In [13]:
!pip install -U yt-dlp

In [38]:
import yt_dlp
import os
import json
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

In [ ]:
def download_videos_with_metadata(url_list):
    extractors = yt_dlp.extractor.gen_extractors()
    supported_sites = [extractor.IE_NAME for extractor in extractors]

    for url in url_list:
        domain = url.split('//')[-1].split('/')[0].split('.')[-2]

        if not any(domain in site for site in supported_sites):
            print(f"Сайт {url} не поддерживается yt_dlp.")
            continue

        if not os.path.exists(domain):
            os.makedirs(domain)

        ydl_opts = {
            'outtmpl': os.path.join(domain, '%(title)s.%(ext)s'),
            'format': 'best',
            'writesubtitles': True,
            'writeinfojson': True,
            'postprocessors': [{
                'key': 'FFmpegVideoConvertor',
                'preferedformat': 'mp4',
            }],
        }

        with yt_dlp.YoutubeDL({'extract_flat': True, 'quiet': True}) as ydl:
            try:
                info_dict = ydl.extract_info(url, download=False)

                # Если это плейлист или страница с несколькими видео, обрабатываем все элементы.
                if 'entries' in info_dict:
                    video_urls = [entry['url'] for entry in info_dict['entries']]
                else:
                    video_urls = [url]

                for video_url in video_urls:
                    try:
                        with yt_dlp.YoutubeDL(ydl_opts) as ydl_download:
                            ydl_download.download([video_url])
                        print(f'Видео и метаданные успешно загружены для {video_url}')
                    except Exception as e:
                        print(f"Ошибка при загрузке {video_url}: {e}")

            except Exception as e:
                print(f"Ошибка при извлечении информации с {url}: {e}")

In [27]:
url_list = [
    'https://www.youtube.com/watch?v=dQw4w9WgXcQ',
    'https://i.imgur.com/rBxB9ih.mp4',
    'https://packaged-media.redd.it/9kr7fm5k16eb1/pb/m2-res_480p.mp4?m=DASHPlaylist.mpd&v=1&e=1690448400&s=a56139280f0853d7bf356fe10e02cf0da081ba56#t=0',
    'https://giphy.com/embed/tsFs4w3ExMwFXVo5JV',
    'https://tenor.com/ru/view/sussa-gif-5432297967573209335',
    'https://test-streams.mux.dev/x36xhzz/x36xhzz.m3u8',
    'https://rutube.ru/video/c6cc4d620b1d4338901770a44b3e82f4/',
    'https://www.youtube.com/feed/trending?bp=6gQJRkVleHBsb3Jl',
]

download_videos_with_metadata(url_list)

[youtube] Extracting URL: https://www.youtube.com/watch?v=dQw4w9WgXcQ
[youtube] dQw4w9WgXcQ: Downloading webpage
[youtube] dQw4w9WgXcQ: Downloading ios player API JSON
[youtube] dQw4w9WgXcQ: Downloading mweb player API JSON
[youtube] dQw4w9WgXcQ: Downloading m3u8 information
[info] dQw4w9WgXcQ: Downloading 1 format(s): 18
[info] There are no subtitles for the requested languages
[info] Writing video metadata as JSON to: youtube/Rick Astley - Never Gonna Give You Up (Official Music Video).info.json
[download] Destination: youtube/Rick Astley - Never Gonna Give You Up (Official Music Video).mp4
[download] 100% of    8.68MiB in 00:00:00 at 24.63MiB/s  
[VideoConvertor] Not converting media file "youtube/Rick Astley - Never Gonna Give You Up (Official Music Video).mp4"; already is in target format mp4
Видео и метаданные успешно загружены для https://www.youtube.com/watch?v=dQw4w9WgXcQ
[Imgur] Extracting URL: https://i.imgur.com/rBxB9ih.mp4
[Imgur] rBxB9ih: Downloading JSON metadata
[Imgur]

ERROR: [rutube] c6cc4d620b1d4338901770a44b3e82f4: No video formats found!; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


Ошибка при извлечении информации с https://rutube.ru/video/c6cc4d620b1d4338901770a44b3e82f4/: ERROR: [rutube] c6cc4d620b1d4338901770a44b3e82f4: No video formats found!; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U
[youtube] Extracting URL: https://www.youtube.com/watch?v=ehVFK-qpA8A
[youtube] ehVFK-qpA8A: Downloading webpage
[youtube] ehVFK-qpA8A: Downloading ios player API JSON
[youtube] ehVFK-qpA8A: Downloading mweb player API JSON
[youtube] ehVFK-qpA8A: Downloading m3u8 information
[info] ehVFK-qpA8A: Downloading 1 format(s): 18
[info] There are no subtitles for the requested languages
[info] Writing video metadata as JSON to: youtube/John Mackk - Pose for me (Visualizer).info.json
[download] youtube/John Mackk - Pose for me (Visualizer).mp4 has already been downloaded
[download] 100% of    7.23MiB
[VideoConvertor] Not converting media file "youtube/John Ma

KeyboardInterrupt: 

In [43]:
def download_videos_from_urls(url_list):
    for url in url_list:
        domain = get_domain_name(url)

        if not os.path.exists(domain):
            os.makedirs(domain)

        video_urls = extract_video_urls_from_page(url)
        if not video_urls:
            print(f"Видео не найдены на странице: {url}")
            continue

        for video_url in video_urls:
            download_video(video_url, domain)

In [44]:
def get_domain_name(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

In [49]:
def extract_video_urls_from_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Ошибка при запросе к {url}: {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    video_urls = []

    for video_tag in soup.find_all("meta", property="og:video"):
        video_url = video_tag.get("content")
        if video_url:
            video_urls.append(video_url)

    for video_tag in soup.find_all("video"):
        source_tag = video_tag.find("source")
        if source_tag and source_tag.get("src"):
            video_urls.append(source_tag["src"])
        elif video_tag.get("src"):
            video_urls.append(video_tag.get("src"))

    for link_tag in soup.find_all("a", href=True):
        href = link_tag["href"]
        if href.endswith(('.mp4', '.webm', '.ogg', '.gif')):
            video_urls.append(href)
    for img_tag in soup.find_all("img", src=True):
        src = img_tag["src"]
        if src.endswith(('.mp4', '.webm', '.ogg', '.gif')):
            video_urls.append(src)

    return video_urls

In [46]:
def download_video(video_url, domain_folder):
    try:
        video_data = requests.get(video_url).content
        video_name = video_url.split('/')[-1]
        video_path = os.path.join(domain_folder, video_name)

        with open(video_path, 'wb') as file:
            file.write(video_data)

        print(f"Видео {video_name} сохранено в {domain_folder}")

    except requests.RequestException as e:
        print(f"Ошибка при загрузке видео с {video_url}: {e}")

In [51]:
url_list = [
    'https://giphy.com/',
    'https://www.youtube.com/',
    'https://tenor.com/ru/',
]

download_videos_from_urls(url_list)

Видео giphy.gif сохранено в giphy.com
Видео 200.gif сохранено в giphy.com
Видео 200.gif сохранено в giphy.com
Видео не найдены на странице: https://www.youtube.com/watch?v=dQw4w9WgXcQ
Видео chhath-puja-happy-chhath-puja.gif сохранено в tenor.com
Видео chhath-puja-happy-chhath-puja.gif сохранено в tenor.com
Видео %E0%A4%9B%E0%A4%81%E0%A4%9F%E0%A4%AE%E0%A4%B9%E0%A4%BE%E0%A4%AA%E0%A4%B0%E0%A5%8D%E0%A4%B5-%E0%A4%89%E0%A4%AA%E0%A4%B5%E0%A4%BE%E0%A4%B8.gif сохранено в tenor.com
Видео event-royal-star.gif сохранено в tenor.com
Видео talking-to-a-wall.gif сохранено в tenor.com
Видео cat-cat-memes.gif сохранено в tenor.com
Видео trumpwinning-trump.gif сохранено в tenor.com
Видео good-morning.gif сохранено в tenor.com
Видео trump-funny.gif сохранено в tenor.com
Видео karnataka-food-karnataka-sweet.gif сохранено в tenor.com
Видео father-ted-dreamy-sleepy-nighty-snoozy-snooze.gif сохранено в tenor.com
Видео %E5%85%83%E6%B0%97-%E3%81%93%E3%82%93%E3%81%AB%E3%81%A1%E3%81%AF.gif сохранено в tenor.com


KeyboardInterrupt: 